# basic file clean up

In [57]:
import sys
sys.path.append('../../')
import glob
import re

import pandas as pd

from config import CLEAN_DATA_DIR, OUTPUT_DIR, RAW_DATA_DIR

from scripts.normalize_data import (
    csv_cleanup,
    update_metadata,
    delete_duplicate_columns,
    change_file_encoding,
    remove_whitespace
)

In [58]:
clean_data_path = CLEAN_DATA_DIR

metadata_file = OUTPUT_DIR/'metadata'/'LIMS'/'Micropal_changes_4.csv' 


## fix bad encoding

In [59]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [60]:
def process_file(path):
    with open(clean_data_path/path, "r") as file:
        changed = False
        try:
            filedata = file.read()
            
        except UnicodeDecodeError as err:
            change_file_encoding(clean_data_path/path)
            changed = True
    return changed
            
change_columns = [process_file(path) for path in metadata['path']]

In [61]:
dict = {"change_file_encoding": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [62]:
new_metadata.to_csv(metadata_file, index=False)

## remove bad characters

In [63]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [64]:
def clean_up_file(path):
    change = False
    with open(clean_data_path/path, "r", errors='ignore') as file:
        filedata = file.read()

        new_filedata = filedata
        new_filedata = new_filedata.replace('�m', 'µm')
        new_filedata = new_filedata.replace('æm', 'µm')

    if new_filedata != filedata:
        with open(clean_data_path/path, "w") as file:
            file.write(new_filedata)
            change = True
            
    return change
            
change_columns = [clean_up_file(file) for file in metadata['path']]


In [65]:
dict = {"remove_bad_characters": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [66]:
new_metadata.to_csv(metadata_file, index=False)

## open and save file

In [67]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [68]:
def process_file(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str) 
    df = csv_cleanup(df, path)
    df.to_csv(path, index=False)
            
change_columns = [process_file(file) for file in metadata['path']]


## remove empty rows

In [69]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [70]:
def process_file(file):
    path = clean_data_path/file
    df = pd.read_csv(path, dtype=str)
    count = len(df)
    
    df.dropna(how='all', axis='rows', inplace=True)
    
    new_count = len(df)        
    changed = count != new_count
    
    if changed:
        df = csv_cleanup(df, path)
        df.to_csv(path, index=False)
        
    return changed
            
change_columns = [process_file(path) for path in metadata['path']]

In [71]:
dict = {"remove_empty_rows": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [72]:
new_metadata.to_csv(metadata_file, index=False)

## remove leading/trailing spaces

In [73]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False


In [74]:
def process_file(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str, header=None)
    dup_df = df.copy()
    
    remove_whitespace(df)
    
    changed = not df.equals(dup_df)
    if changed:
        df.to_csv(path, index=False, header=False)

    return changed
            
change_columns = [process_file(file) for file in metadata['path']]


In [75]:
dict = {"remove_spaces": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters,remove_spaces
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False,True
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False,True
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False,True
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False,True
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False,True


In [76]:
new_metadata.to_csv(metadata_file, index=False)

## remove duplicate rows

In [77]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters,remove_spaces
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False,True
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False,True
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False,True
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False,True
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False,True


In [78]:
def delete_duplicate_rows(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str)
    
    columns = df.columns
    df = df.drop_duplicates()
    new_columns = df.columns
    
    changed = len(columns) != len(new_columns)
    
    if changed:
        df = csv_cleanup(df, path)
        df.to_csv(path, index=False)

    return changed
    
change_columns = [delete_duplicate_rows(file) for file in metadata['path']]


In [79]:
dict = {"delete_duplicate_rows": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters,remove_spaces
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False,True
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False,True
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False,True
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False,True
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False,True


In [80]:
new_metadata.to_csv(metadata_file, index=False)

## remove renamed duplicate column names with same values

In [81]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters,remove_spaces
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False,True
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False,True
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False,True
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False,True
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False,True


In [82]:
def delete_columns(file):
    path = f"{clean_data_path}/{file}"
    df = pd.read_csv(path, dtype=str)
    
    columns = df.columns
    delete_duplicate_columns(df)
    new_columns = df.columns
    
    changed = len(columns) != len(new_columns)

    if changed:
        df = csv_cleanup(df, path)
        df.to_csv(path, index=False)

    return changed
    
change_columns = [delete_columns(file) for file in metadata['path']]


In [83]:
dict = {"delete_duplicate_columns": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,path,taxon_group,change_file_encoding,remove_empty_rows,remove_spaces_from_columns,delete_duplicate_colums_with_spaces,delete_duplicate_rows,delete_renamed_duplicate_columns,remove_bad_characters,remove_spaces,delete_duplicate_columns
0,320_U1334_benthic_forams.csv,LIMS/Micropal_CSV_4/320_U1334_benthic_forams.csv,benthic_forams,False,False,False,False,False,False,False,True,False
1,361_U1479B_nannofossils.csv,LIMS/Micropal_CSV_4/361_U1479B_nannofossils.csv,nannofossils,False,False,False,False,False,False,False,True,False
2,372_U1517C_planktic_forams.csv,LIMS/Micropal_CSV_4/372_U1517C_planktic_forams...,planktic_forams,False,False,False,False,False,False,False,True,False
3,317_U1352_planktic_forams.csv,LIMS/Micropal_CSV_4/317_U1352_planktic_forams.csv,planktic_forams,False,False,True,False,False,False,False,True,False
4,323_U1339_palynology.csv,LIMS/Micropal_CSV_4/323_U1339_palynology.csv,palynology,False,False,False,False,False,False,False,True,False


In [84]:
new_metadata.to_csv(metadata_file, index=False)